# Sistemas Inteligentes 2021/2022

## Mini-projeto 2: Quadrados Latinos


## Grupo: 42

### Elementos do Grupo

Número:  53687       Nome:   Ariana Dias  
Número:  53746       Nome:   Andrei Tataru  
Número:  51127       Nome:   Luís Ferreirinha  

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

## Representação de variáveis, domínios, vizinhos e restrições

Descreva aqui, textualmente, como decidiu representar as variáveis, domínios, vizinhos e restrições em Python;

Considerando a representação do quadrado como uma matriz _n x n_,
- Variáveis:
    - Representar por uma lista de coordenadas;
- Domínios:
    - Conjunto de números {1,....,n}
- Vizinhos:
    - Dicionário que define o grafo de restrições ({(i,j):[todos os elementos da linha i e todos os elementos da coluna j]})
- Restrições:
    - Todos os elementos da linha i são diferentes, todos os elementos da coluna j são diferentes.


## Formulação do problema

In [3]:
from csp import *

def quadrado_latino(n=3, initial={}) -> CSP:
    """
    Pode receber parametros ou não.
    Deve devolver um CSP, à semelhança dos guiões das aulas PL.
    Comente o código.
    """
    variaveis = {(i,j) for j in range(0,n) for i in range(0,n)}

    conjunto = {i for i in range(1,n+1)}
    
    dominios = initial

    # Diferenca dos dois conjuntos vai resultar nas variaveis nao preenchidas
    for item in set(variaveis - initial.keys()):
            dominios[item] = conjunto
    
    condicao_vizinhanca = lambda x,y : (x[0] == y[0] and x[1] != y[1]) or (x[0] != y[0] and x[1] == y[1])
    
    vizinhos = {var : set(filter(lambda x: condicao_vizinhanca(x,var), variaveis)) for var in variaveis}

    def restricoes(X, a, Y, b):
        if Y in vizinhos[X]:
            return a != b
        return True
        
    return CSP(variaveis, dominios, vizinhos, restricoes)

## Criação do problema do quadrado latino simples

Mostrem que o código está a funcionar, construindo um problema de quadrado latino *4x4*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Mostre como podemos criar um puzzle com quadrados já preenchidos, e qual o impacto que isso tem nas variáveis, domínios iniciais, e vizinhos.

In [24]:
p2 = quadrado_latino(4)
print("Variáveis = ", p2.variables)
print("Domínios = ", p2.domains)
print("Vizinhos = ", p2.neighbors)

Variáveis =  {(0, 1), (1, 2), (2, 1), (0, 0), (3, 1), (1, 1), (0, 3), (2, 0), (3, 0), (2, 3), (0, 2), (3, 3), (2, 2), (1, 0), (3, 2), (1, 3)}
Domínios =  {(0, 1): {1, 2, 3, 4}, (1, 2): {1, 2, 3, 4}, (2, 1): {1, 2, 3, 4}, (3, 1): {1, 2, 3, 4}, (0, 2): {1, 2, 3, 4}, (2, 2): {1, 2, 3, 4}, (1, 0): {1, 2, 3, 4}, (3, 2): {1, 2, 3, 4}, (1, 3): {1, 2, 3, 4}, (0, 0): {1, 2, 3, 4}, (1, 1): {1, 2, 3, 4}, (0, 3): {1, 2, 3, 4}, (2, 0): {1, 2, 3, 4}, (3, 0): {1, 2, 3, 4}, (2, 3): {1, 2, 3, 4}, (3, 3): {1, 2, 3, 4}}
Vizinhos =  {(0, 1): {(2, 1), (0, 0), (3, 1), (1, 1), (0, 3), (0, 2)}, (1, 2): {(1, 1), (0, 2), (2, 2), (1, 0), (3, 2), (1, 3)}, (2, 1): {(0, 1), (3, 1), (1, 1), (2, 0), (2, 3), (2, 2)}, (0, 0): {(0, 1), (0, 3), (2, 0), (3, 0), (0, 2), (1, 0)}, (3, 1): {(0, 1), (2, 1), (1, 1), (3, 0), (3, 3), (3, 2)}, (1, 1): {(0, 1), (1, 2), (2, 1), (3, 1), (1, 0), (1, 3)}, (0, 3): {(0, 1), (0, 0), (2, 3), (0, 2), (3, 3), (1, 3)}, (2, 0): {(2, 1), (0, 0), (3, 0), (2, 3), (2, 2), (1, 0)}, (3, 0): {(0, 0),

Para criar um puzzle com quadrados já preenchidos, alteramos a função de formulação do problema, fazendo com que esta agora receba também um dicionário com os quadrados já preenchidos.

O facto de o quadrado ser semi preenchido apenas vai afetar os dominios iniciais, fazendo com que o dominio inicial das variaveis ja com valores atribuidos seja reduzido para essas.


In [18]:
preenchido = {(0,0): [1], (1,1):[3], (2,2):[2]}

pSP = quadrado_latino(3, preenchido)
print("Variáveis = ", pSP.variables)
print("Domínios = ", pSP.domains)
print("Vizinhos = ", pSP.neighbors)

r = backtracking_search(pSP)
print(r)

Variáveis =  {(0, 1), (1, 2), (2, 1), (0, 0), (1, 1), (2, 0), (0, 2), (2, 2), (1, 0)}
Domínios =  {(0, 0): [1], (1, 1): [3], (2, 2): [2], (0, 1): {1, 2, 3}, (0, 2): {1, 2, 3}, (1, 2): {1, 2, 3}, (2, 1): {1, 2, 3}, (1, 0): {1, 2, 3}, (2, 0): {1, 2, 3}}
Vizinhos =  {(0, 1): {(1, 0), (1, 2), (2, 0), (2, 2)}, (1, 2): {(0, 1), (2, 0), (2, 1), (0, 0)}, (2, 1): {(1, 0), (0, 2), (1, 2), (0, 0)}, (0, 0): {(1, 1), (1, 2), (2, 1), (2, 2)}, (1, 1): {(2, 2), (0, 2), (2, 0), (0, 0)}, (2, 0): {(0, 1), (1, 1), (1, 2), (0, 2)}, (0, 2): {(1, 0), (1, 1), (2, 0), (2, 1)}, (2, 2): {(0, 1), (1, 0), (1, 1), (0, 0)}, (1, 0): {(0, 1), (0, 2), (2, 1), (2, 2)}}
{(0, 1): 1, (1, 2): 3, (2, 1): 2, (0, 0): 1, (1, 1): 3, (2, 0): 2, (0, 2): 1, (2, 2): 2, (1, 0): 3}


Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística.

In [25]:
import timeit
tempos = {}
n = 3

In [39]:
QuadradoLatinoSemInferência = quadrado_latino(n)
start = timeit.default_timer()
r = backtracking_search(QuadradoLatinoSemInferência)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoSemInferência) = ',r)
tempos['QuadradoLatinoSemInferência'] = stop - start

Time:  0.0007900999917183071
Assignment (QuadradoLatinoSemInferência) =  {(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 2, (1, 1): 3, (2, 1): 1, (0, 2): 3, (1, 2): 1, (2, 2): 2}


In [40]:
QuadradoLatinoComInferência_FRWDchecking = quadrado_latino(n)
start = timeit.default_timer()
r = backtracking_search(QuadradoLatinoComInferência_FRWDchecking, inference=forward_checking)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComInferência_FRWDchecking) = ',r)
tempos['QuadradoLatinoComInferência_FRWDchecking'] = stop - start

Time:  0.0006933000113349408
Assignment (QuadradoLatinoComInferência_FRWDchecking) =  {(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 2, (1, 1): 3, (2, 1): 1, (0, 2): 3, (1, 2): 1, (2, 2): 2}


In [41]:
QuadradoLatinoComAC3 = quadrado_latino(n)
start = timeit.default_timer()
p_AC3 = AC3(QuadradoLatinoComAC3)
r = backtracking_search(p_AC3)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComAC3) = ',r)
tempos['QuadradoLatinoComAC3'] = stop - start

Time:  0.0010866000084206462
Assignment (QuadradoLatinoComAC3) =  {(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 2, (1, 1): 3, (2, 1): 1, (0, 2): 3, (1, 2): 1, (2, 2): 2}


In [42]:
QuadradoLatinoComInferência_MAC = quadrado_latino(n)
start = timeit.default_timer()
r = backtracking_search(QuadradoLatinoComInferência_MAC, inference=mac)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComInferência_MAC) = ',r)
tempos['QuadradoLatinoComInferência_MAC'] = stop - start

Time:  0.001676399988355115
Assignment (QuadradoLatinoComInferência_MAC) =  {(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 2, (1, 1): 3, (2, 1): 1, (0, 2): 3, (1, 2): 1, (2, 2): 2}


In [43]:
QuadradoLatinoComAC3eInferência_MAC = quadrado_latino(n)
start = timeit.default_timer()
p_AC3 = AC3(QuadradoLatinoComAC3eInferência_MAC)
r = backtracking_search(p_AC3, inference=mac)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComAC3eInferência_MAC) = ',r)
tempos['QuadradoLatinoComAC3eInferência_MAC'] = stop - start

Time:  0.0015719000075478107
Assignment (QuadradoLatinoComAC3eInferência_MAC) =  {(0, 0): 1, (1, 0): 2, (2, 0): 3, (0, 1): 2, (1, 1): 3, (2, 1): 1, (0, 2): 3, (1, 2): 1, (2, 2): 2}


In [44]:
QuadradoLatinoComHeuristica = quadrado_latino(n)
start = timeit.default_timer()
r = backtracking_search(QuadradoLatinoComHeuristica, select_unassigned_variable = mrv)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComHeuristica) = ',r)
tempos['QuadradoLatinoComHeuristica'] = stop - start

Time:  0.001438899984350428
Assignment (QuadradoLatinoComHeuristica) =  {(0, 2): 1, (1, 2): 2, (0, 0): 2, (1, 0): 3, (2, 1): 2, (2, 0): 1, (2, 2): 3, (1, 1): 1, (0, 1): 3}


In [45]:
QuadradoLatinoComHeuristicaComInferencia = quadrado_latino(n)
start = timeit.default_timer()
r = backtracking_search(QuadradoLatinoComHeuristicaComInferencia, select_unassigned_variable = mrv, inference=forward_checking)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Assignment (QuadradoLatinoComHeuristicaComInferencia) = ',r)
tempos['QuadradoLatinoComHeuristicaComInferencia'] = [stop - start]

Time:  0.0006595999875571579
Assignment (QuadradoLatinoComHeuristicaComInferencia) =  {(2, 1): 1, (2, 2): 2, (2, 0): 3, (1, 0): 1, (1, 2): 3, (0, 0): 2, (0, 2): 1, (1, 1): 2, (0, 1): 3}


In [48]:
for item in tempos:
    print(item, tempos[item])

print(tempos.keys())

QuadradoLatinoSemInferência 0.0007900999917183071
QuadradoLatinoComInferência_FRWDchecking 0.0006933000113349408
QuadradoLatinoComAC3 0.0010866000084206462
QuadradoLatinoComInferência_MAC 0.001676399988355115
QuadradoLatinoComAC3eInferência_MAC 0.0015719000075478107
QuadradoLatinoComHeuristica 0.001438899984350428
QuadradoLatinoComHeuristicaComInferencia 0.0006595999875571579
dict_keys(['QuadradoLatinoSemInferência', 'QuadradoLatinoComInferência_FRWDchecking', 'QuadradoLatinoComAC3', 'QuadradoLatinoComInferência_MAC', 'QuadradoLatinoComAC3eInferência_MAC', 'QuadradoLatinoComHeuristica', 'QuadradoLatinoComHeuristicaComInferencia'])


In [47]:
import pandas as pd
temposSorted = dict(sorted(tempos.items(), key=lambda x:x[1]))
pd.DataFrame(temposSorted.values(),index = temposSorted.keys(), columns= [n])

,3
QuadradoLatinoComHeuristicaComInferencia,0.000660
QuadradoLatinoComInferência_FRWDchecking,0.000693
QuadradoLatinoSemInferência,0.000790
QuadradoLatinoComAC3,0.001087
QuadradoLatinoComHeuristica,0.001439
QuadradoLatinoComAC3eInferência_MAC,0.001572
QuadradoLatinoComInferência_MAC,0.001676


## Big Duvidas AQUI

Com a tabela acima é possível concluir que:
- o algoritmo que demora menos tempo a encontrar uma solução é a **procura com Retrocesso usando a inferência Forward Checking**, demora apenas 0.00046 segundos;
- a solução do problema utilizando a procura com retrocesso sem inferências nem heurísticas demora mais tempo (0.000991 segundos) que a procura com retrocesso depois de pré-processamento com recurso ao algoritmo **AC3** (0.000890 segundos);
- a procura com retrocesso usando Inferência e Heuristica (0.000997 segundos) é mais lenta que a procura sem (0.000991 segundos), sendo esta diferença na ordem dos 10 microsegundos;
- independentemente da realização de um pré-processamento com AC3, a procura com inferência em que se mantém a consistência (mac), demora mais tempo, diferença na ordem dos 10 milisegundos;

## Visualização do problema

In [28]:
import numpy as np

def visualizacao(n, variaveis, solucao, preenchidos = {}):
    def getList(dict):
        list = []
        for key in dict.keys():
            list.append(key)
        return list

    coor_preenchidas = getList(preenchidos)
    coordenadas = coor_preenchidas + variaveis
    matrix = np.zeros((n,n))
    for tuple in coordenadas:
        if tuple not in variaveis:
            matrix[tuple[0]][tuple[1]] = preenchido[tuple]

    print('Antes: \n', matrix)
    for tuple in solucao.keys():
            matrix[tuple[0]][tuple[1]] = solucao[tuple]
    print('Depois: \n', matrix)

ModuleNotFoundError: No module named 'numpy'

In [32]:
visualizacao(6, quadrado_latino(6).variables, backtracking_search(quadrado_latino(6)))

Antes: 
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Depois: 
 [[1. 2. 1. 2. 1. 2.]
 [2. 1. 2. 1. 2. 1.]
 [1. 2. 1. 2. 1. 2.]
 [2. 1. 2. 1. 2. 1.]
 [1. 2. 1. 2. 1. 2.]
 [2. 1. 2. 1. 2. 1.]]


In [27]:
pSP.display()

TypeError: display() missing 1 required positional argument: 'assignment'

## Criação do problema Futoshiki *5x5*

Mostrem que o código está a funcionar, construindo um problema de Futoshiki *5x5*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Utilize o [link](https://www.futoshiki.org/) para gerar puzzles e validar a implementação.

In [24]:
def maior(x,y):
    return x > y

def menor(x,y):
    return x < y

def futoshiki(n=3, initial={}, desigualdades={}):
    """
    Pode receber parametros ou não.
    Deve devolver um CSP, à semelhança dos guiões das aulas PL.
    Comente o código.
    """
    variaveis = {(i,j) for j in range(0,n) for i in range(0,n)}

    conjunto = {i for i in range(1,n+1)}
    
    dominios = initial

    # Diferenca dos dois conjuntos vai resultar nas variaveis nao preenchidas
    for item in set(variaveis - initial.keys()):
            dominios[item] = conjunto
    
    condicao_vizinhanca = lambda x,y : (x[0] == y[0] and x[1] != y[1]) or (x[0] != y[0] and x[1] == y[1])
    
    vizinhos = {var : set(filter(lambda x: condicao_vizinhanca(x,var), variaveis)) for var in variaveis}

    def restricoes(X, a, Y, b):
        if (X,Y) in desigualdades.keys():
            if not desigualdades[(X,Y)](a,b):
                return False
        if (Y,X) in desigualdades.keys():
            if not desigualdades[(Y,X)](b,a):
                return False
        if Y in vizinhos[X]:
            return  a != b
        return True



    return CSP(variaveis, dominios, vizinhos, restricoes)

In [22]:
preenchidos = {(1,2) : {1}}
desigualdades = {((1,0),(2,0)) : menor, ((2,0),(3,0)) : menor, ((2,3),(2,2)): maior}

In [25]:
p = futoshiki(5, preenchidos, desigualdades)
print("Variáveis = ", p.variables)
print("Domínios = ", p.domains)
print("Vizinhos = ", p.neighbors)
r = backtracking_search(p)

Variáveis =  {(4, 0), (3, 4), (4, 3), (3, 1), (0, 2), (2, 2), (1, 0), (1, 3), (4, 2), (3, 0), (3, 3), (0, 1), (2, 4), (1, 2), (0, 4), (2, 1), (3, 2), (4, 1), (4, 4), (0, 0), (1, 1), (0, 3), (2, 0), (1, 4), (2, 3)}
Domínios =  {(1, 2): {1}, (0, 1): {1, 2, 3, 4, 5}, (2, 4): {1, 2, 3, 4, 5}, (4, 0): {1, 2, 3, 4, 5}, (0, 4): {1, 2, 3, 4, 5}, (3, 4): {1, 2, 3, 4, 5}, (2, 1): {1, 2, 3, 4, 5}, (4, 3): {1, 2, 3, 4, 5}, (3, 1): {1, 2, 3, 4, 5}, (0, 2): {1, 2, 3, 4, 5}, (2, 2): {1, 2, 3, 4, 5}, (1, 0): {1, 2, 3, 4, 5}, (3, 2): {1, 2, 3, 4, 5}, (1, 3): {1, 2, 3, 4, 5}, (4, 1): {1, 2, 3, 4, 5}, (4, 4): {1, 2, 3, 4, 5}, (0, 0): {1, 2, 3, 4, 5}, (1, 1): {1, 2, 3, 4, 5}, (0, 3): {1, 2, 3, 4, 5}, (2, 0): {1, 2, 3, 4, 5}, (4, 2): {1, 2, 3, 4, 5}, (3, 0): {1, 2, 3, 4, 5}, (1, 4): {1, 2, 3, 4, 5}, (2, 3): {1, 2, 3, 4, 5}, (3, 3): {1, 2, 3, 4, 5}}
Vizinhos =  {(4, 0): {(4, 4), (4, 3), (0, 0), (2, 0), (4, 2), (3, 0), (1, 0), (4, 1)}, (3, 4): {(4, 4), (2, 4), (0, 4), (3, 1), (1, 4), (3, 0), (3, 3), (3, 2)},

In [29]:
print(r)

{(4, 0): 1, (3, 4): 1, (4, 3): 2, (3, 1): 2, (0, 2): 2, (2, 2): 4, (1, 0): 2, (1, 3): 4, (4, 2): 3, (3, 0): 4, (3, 3): 3, (0, 1): 3, (2, 4): 2, (1, 2): 1, (0, 4): 4, (2, 1): 1, (3, 2): 5, (4, 1): 4, (4, 4): 5, (0, 0): 5, (1, 1): 5, (0, 3): 1, (2, 0): 3, (1, 4): 3, (2, 3): 5}


Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística. Até que dimensão consegue resolver o problema em menos de 1 minuto?

In [ ]:
# código 

## Visualização do problema

In [ ]:
# Implemente uma função que permita visualizar o puzzle Futoshiki, antes e depois de resolvido. Compare com a solução obtida pelo seu algoritmo. 
# No caso de não implementar esta função, inclua um screenshot do problema e da sua solução.